IMPORTS

In [119]:
# tabula para ler tabelas de pdf
# pandas para usar dataframes
# numpy  para trabalhar com numeros
import tabula as tb
import pandas as pd
import numpy  as np

VARIÁVEIS

In [120]:
#local do arquivo a ser lido
FILE_PATH = "RHPP05LA - GERAL.pdf"

# normalizacao de dados
TEXTOS_NORMALIZADOS_JAVA = {
    "1/3 Férias Total Férias": "13_ferias total_ferias", 
    "1/3 Férias"            : "13_ferias", 
    "Total Férias"          : "total_ferias", 
    "13o Salário"           : "13_salario", 
    "TOTAL"                 : "total", 
    "Apropriação"           : "apropriacao", 
    "INSS 20,00%"           : "inss", 
    ".*Ac. Trab. .*"        : "ac_trab", 
    "FGTS 8%"               : "fgts", 
    "SP-PREVCOM"            : "spprevcom", 
    "Total de Encargos"     : "total_encargos", 
    "Férias em pecúnia"     : "ferias_pecunia", 
    "SPPREV"                : "spprev", 
    "Férias"                : "ferias", 
    "\."                    : "",
    ","                     : ".",
    "APROPRIAÇÃO MENSAL"    : "apropriado",
    "REALIZADO MENSAL"      : "realizado",
    "SALDO FINAL MENSAL"    : "saldo",
    "REVERSÃO / REVISÃO DAS APROPRIAÇÕES" : "revertido",
    }

# renomeacao de colunas do dataframe
COLUNAS_RENOMEADAS = {
    "Unnamed: 0": "tipo", 
    "Unnamed: 1":"Col1", 
    "Col1":"Col2", 
    "Col2":"Col3", 
    "Unnamed: 2":"Col4", 
    "Unnamed: 3":"Col5",
    }

# reordenacao de colunas do dataframe
COLUNAS_ORDENADAS = [
    "Unnamed: 0", 
    "Unnamed: 1", 
    "Col1", 
    "Col2", 
    "Unnamed: 2", 
    "Unnamed: 3", 
    ]

# tipos de tabelas do pdf
TIPOS_TABELAS =  [
    "CLT", 
    "AUTÁRQUICO", 
    ]

# tipos de tabelas do pdf que nao vao ser usadas
TIPOS_TABELAS_A_IGNORAR = [
    "TOTAIS", 
    ]

# tipos de tabelas das tabelas
TABELAS_INTERNAS = [
    "apropriado", 
    "realizado", 
    "saldo", 
    "revertido", 
]

FUNÇÕES

In [121]:
def normalize_text_java(tables:list) -> list:
    for index, table in enumerate(tables, start=1):
        tables[index-1] = table.replace(TEXTOS_NORMALIZADOS_JAVA, inplace=False, regex=True)
    return tables

def get_table_type(table:list) -> str:
    if not any(ignore_type in column for column in table for ignore_type in TIPOS_TABELAS_A_IGNORAR):
        for type in TIPOS_TABELAS:
            if type in table.columns:
                return type
    return None

def split_column(table:list, type:str, col1="Col1", col2="Col2") -> pd.DataFrame:
    df = pd.DataFrame(columns=[col1, col2])
    for value in table[type]:
        values = value.split()
        df.loc[len(df)] = [values[0], values[1] if len(values) > 1 else None]
    return df

def join_columns(table:list, df:pd.DataFrame, type:str) -> pd.DataFrame:
    table.drop(columns=[type], inplace=True)
    table = pd.concat([table, df], axis=1)
    return table

def format_data_frame(df:pd.DataFrame, type:str)->pd.DataFrame:
    df = df.reindex(columns=COLUNAS_ORDENADAS)
    COLUNAS_RENOMEADAS["Unnamed: 0"] = type
    df = df.rename(columns=COLUNAS_RENOMEADAS)
    return df

def get_line_positions(df:pd.DataFrame, col:str="Col2") -> list:
    positions = []
    for index, line in enumerate(df[col]):
        if line in TABELAS_INTERNAS:
            positions.append(index)
    positions.append(len(df))
    return positions

def split_internal_dataframe(df_main:pd.DataFrame, pos:list):
    results = []
    for i in range(len(pos) - 1):
            df_new  = df_main[pos[i] : pos[i+1]]
            type_df = df_new.iloc[0,2]

            df_new  = df_new.rename(columns=df_new.iloc[1].replace(np.nan, ""))[2:]
            df_new.reset_index(drop=True, inplace=True)
            
            results.append((type_df, df_new))
    return results

def get_dataframes(df_main:pd.DataFrame) -> list:
    type = get_table_type(df_main)
    if type is not None:
        df_main = join_columns(df_main, split_column(df_main, type), type)
        df_main = format_data_frame(df_main, type)
        return split_internal_dataframe(df_main, get_line_positions(df_main))
    else:
        return []

EXECUÇÃO PRINCIPAL

In [122]:

tabelas = normalize_text_java(tb.read_pdf(FILE_PATH, pages="all"))

for tabela in tabelas:
    resultados = get_dataframes(tabela)
    for tipo_tabela, sub_tabela in resultados:
        print(tipo_tabela)
        display(sub_tabela)

TypeError: 'NoneType' object is not iterable